### RAG

In [2]:
from getpass import getpass

from openai import api_key, base_url
%pip install -qU langchain-openai langchain langchain_community langchainhub
%pip install -qU chromadb==0.5.3

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain import hub as langchain_hub
from langchain.schema import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain_openai import OpenAIEmbeddings
import os
#from langchain_chroma import Chroma
from langchain_community.vectorstores.chroma import Chroma
from langchain_core.prompts import PromptTemplate
from string import Template
import uuid

In [ ]:
# read ./data/data.md as knowledge base
file_path = os.path.join('data', 'data.md')
with open(file_path, 'r', encoding='utf-8') as file:
    docs_string = file.read()

# split the document into chunks base on markdown headers.
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
text_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
splits = text_splitter.split_text(docs_string)
print("Length of splits: " + str(len(splits)))
print(splits)

In [5]:
from getpass import getpass

random_directory = "./" + str(uuid.uuid4())

key = getpass()

# embed doc & persist to directory
embedding = OpenAIEmbeddings(
    model="text-embedding-3-small", 
    openai_api_key=key,
    openai_api_base="https://api.apiyi.com/v1")
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding, persist_directory=random_directory)
# LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
# vectorstore.persist()

In [4]:
# rag
retriever = vectorstore.as_retriever()

# prompt template
prompt_template = """
            Use context below to answer the questions.
            If u don't know the answer, just answer u don't know, please don't try to compile it.
            As concise as possible, at most 3 sentences.
            In the end, please say "Thanks for your question.:)"
            
            {context}

            Question: {question}

            Helpful Answer:
            """
custom_rag_prompt = PromptTemplate.from_template(prompt_template)

def format_docs(docs):
    print("matched doc: \n", "\n\n".join(doc.page_content for doc in docs))
    return "\n\n".join(doc.page_content for doc in docs)

llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=key,
    base_url="https://api.apiyi.com/v1"
)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)


# rag is not able to answer
res = rag_chain.invoke("Who is in charge of the most systems?")
print("\n\nLLM Answer: ", res)

NameError: name 'vectorstore' is not defined

### Agent RAG

In [7]:
# search by similarity, where k is the number of searched block. k++ per iteration
def search_docs(query, k=1):
    results = vectorstore.similarity_search_with_score(
        query,
        k=k,
    )
    return "\n\n".join(doc.page_content for doc, score in results)

user_query = "Who is in charge of the most systems?"

# context = matched block(s) from vector store
check_can_answer_system_prompt = """
                                Check if LLM is able to answer the question, if not，return JSON string "{"can_answer": false}"，otherwise return JSON string "{"can_answer": true}"。
                                context：\n $context
                                question：$question
                                """

llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=key,
    base_url="https://api.apiyi.com/v1"
)
k = 1
docs = ""

while True:
    print("Iteration: ", k)
    if k > 15:
        break
        
    docs = search_docs(user_query, k)
    print("matched doc: ", docs)
    
    # build template from prompt & render by key
    template = Template(check_can_answer_system_prompt)
    filled_prompt = template.substitute(context=docs, question=user_query)
    
    # build msg by prompt
    # check if llm is able to answer the question
    messages = [
        (
            "system",
            filled_prompt,
        ),
        (
            "human", 
            "start to check if context is able to answer the question."
        ),
    ]
    
    # invoke llm
    llm_message = llm.invoke(messages)
    print("\nLLM Ans: ", llm_message.content, "\n")
    
    if llm_message.content == '{"can_answer": true}':
        break
    # k++ per iteration
    else:
        k += 1

print("Knowledge base is matched, start to answer the question.\n")
final_system_prompt = """
                    Use context below to answer the questions.
                    If u don't know the answer, just answer u don't know, please don't try to compile it.
                    As concise as possible, at most 3 sentences.
                    In the end, please say "Thanks for your question.:)
                    """
    
final_messages = [
     (
        "system",
        final_system_prompt,
     ),
    ("human", "context: \n"+ docs +"\nquestion: " + user_query),
]
llm_message = llm.invoke(final_messages)
print("\nLLM Final Ans: ", llm_message.content, "\n")

Iteration:  1
matched doc:  - **Access Control**: Strictly control system access permissions and implement the principle of least privilege.
- **Data Encryption**: Encrypt sensitive data to protect user privacy.
- **Security Audits**: Conduct regular security audits to check for potential security risks.

LLM Ans:  {"can_answer": false} 

Iteration:  2
matched doc:  - **Access Control**: Strictly control system access permissions and implement the principle of least privilege.
- **Data Encryption**: Encrypt sensitive data to protect user privacy.
- **Security Audits**: Conduct regular security audits to check for potential security risks.

- **Data Backup**: Regularly back up databases and important files to ensure data security.
- **System Updates**: Regularly update systems and software to fix known vulnerabilities.
- **Performance Tuning**: Periodically check system performance and make necessary optimizations.

LLM Ans:  {"can_answer": false} 

Iteration:  3


KeyboardInterrupt: 